In [ ]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [1]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [18]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "NLbFIicW1Xj2C5LgMsrhDdwGW"
    consumer_secret = "RwSSpg4t0WktJEOb8nYAjWpbwPMZIopUY1HUB6LGaeqCUSLHO8"
    access_token = "1517636929717608448-1K1qkuZ1r8hUuaJkrbQpejGKiceZDx"
    access_token_secret = "JciruqpsM57rjn15G6linGw6rIiXfFbNB0uNMRuSQAgek"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)
    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
                 -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii
    stream_listener = StreamListener(time_limit=800, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(languages=['en'], encoding="utf-8", track=["Elon"])



{'id': 1518717618332393472, 'username': 'ThuPlug', 'created_at': 'Mon Apr 25 22:24:31 +0000 2022', 'lng': 139.65925049999998, 'lat': 35.7060655, 'text': 'Elon really bought Twitter 😳'}
{'id': 1518717643175469061, 'username': 'Patrici20765403', 'created_at': 'Mon Apr 25 22:24:37 +0000 2022', 'lng': -89.5960515, 'lat': 40.6446025, 'text': "I don't worship anyone but God"}
{'id': 1518717645440258048, 'username': 'MileHighBrendan', 'created_at': 'Mon Apr 25 22:24:37 +0000 2022', 'lng': -104.92209349999999, 'lat': 39.713563, 'text': 'If Donald Trump ever called me a “good man,” I would have to immediately re-evaluate my life.That’s like Tommy Wi… https://t.co/A7pXOnder6'}
{'id': 1518717663966547970, 'username': 'FloridaBengals', 'created_at': 'Mon Apr 25 22:24:42 +0000 2022', 'lng': -82.67290299999999, 'lat': 27.7957195, 'text': "If Elon Musk's goal was to buy a crappy company that used to be great a long time ago, he should've bought the Reds instead."}
{'id': 1518717673240104960, 'usernam

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
